In [2]:
import json

with open('/Users/sunnyyu/Desktop/research/children_speech_books/code/python/generics/MERGED_DATA_SPEECH.json', 'r') as file:
    generics_data = json.load(file)

In [10]:
len(generics_data)

360341

In [15]:
import json
import re
import tqdm as tqdm

def categorize_sentence_by_subject(sentence, subject):
    # Augmented lists of universal and existential quantifiers
    universal_quantifiers = [
        "all", "every", "each", "any", "no", "none", "never", "always", 
        "everybody", "everyone", "everything", "everywhere", "no one", 
        "nobody", "nothing", "neither", "every single", "without exception", 
        "entire", "whole", "everywhere", "whenever"
    ]
    
    existential_quantifiers = [
        "some", "a", "an", "one", "several", "many", "few", "a few", 
        "a couple of", "at least one", "a number of", "a lot of", 
        "a majority of", "a minority of", "certain", "most", "almost all", 
        "more than half", "half of", "less than half", "two-thirds", 
        "three-quarters", "dozens of", "hundreds of", "thousands of", 
        "millions of", "billions of", "about", "approximately", "various", 
        "plenty of", "somebody", "someone", "something", "somewhere", 
        "multiple", "varied", "part of", "enough", "much", 
        "a portion of", "particular", "certain individuals", "occasional"
    ]
    
    # Normalize the sentence and subject to lowercase for easier matching
    sentence = sentence.lower()
    subject = subject.lower()
    
    # Create a pattern to match quantifiers directly before the subject
    universal_pattern = r'\b(?:' + '|'.join(map(re.escape, universal_quantifiers)) + r')\b\s+\b' + re.escape(subject) + r'\b'
    existential_pattern = r'\b(?:' + '|'.join(map(re.escape, existential_quantifiers)) + r')\b\s+\b' + re.escape(subject) + r'\b'
    
    # Check if any universal quantifier appears directly before the subject
    if re.search(universal_pattern, sentence):
        return "Universal Quantifier"
    
    # Check if any existential quantifier appears directly before the subject
    if re.search(existential_pattern, sentence):
        return "Existential Quantifier"
    
    # If no quantifiers are found directly before the subject
    return "Neither"

def process_json_file(data, output_file):
    
    # Process each book
    for book in data:
        for sentence_data in book:
            # Get the sentence and subject from the sentence data
            sentence = sentence_data.get('sentence', "")
            subjects = sentence_data.get('subjects', [])
            if subjects:
                # Assume the first subject is the main one (you can adjust if needed)
                subject_text = subjects[0].get('subject', "")
                # Categorize the sentence based on the subject
                classification = categorize_sentence_by_subject(sentence, subject_text)
                # Add the classification to the sentence data
                sentence_data['quantified_classification'] = classification
            else:
                # If no subject is present, classify as "Neither"
                sentence_data['quantified_classification'] = "Neither"
    
    # Save the updated data to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)


output_file = 'your_output_file.json'
process_json_file(generics_data, output_file)


AttributeError: 'str' object has no attribute 'get'

In [3]:
import json
import re
from tqdm import tqdm
from nltk.corpus import stopwords

# Combine the original excluded pronouns with NLTK's stop words
excluded_pronouns = {
    'he', 'you', 'we', 'she', 'i', 'they', 'him', 'her', 'them', 'his', 'hers', 'theirs', 'it', 'way', 'thing'
}
excluded_pronouns.update(set(stopwords.words('english')))


def categorize_sentence_by_subject(sentence, subject, universal_pattern, existential_pattern):
    # Normalize the sentence and subject to lowercase for easier matching
    sentence = sentence.lower()
    subject = subject.lower()

    # Check if any universal quantifier appears directly before the subject
    if re.search(universal_pattern, sentence):
        return "Universal Quantifier"
    
    # Check if any existential quantifier appears directly before the subject
    if re.search(existential_pattern, sentence):
        return "Existential Quantifier"
    
    # If no quantifiers are found directly before the subject
    return "Neither"

def process_json_file(data, output_file):
    # Augmented lists of universal and existential quantifiers
    universal_quantifiers = [
        "all", "every", "each", "any", "no", "none", "never", "always", 
        "everybody", "everyone", "everything", "everywhere", "no one", 
        "nobody", "nothing", "neither", "every single", "without exception", 
        "entire", "whole", "everywhere", "whenever"
    ]
    
    existential_quantifiers = [
        "some", "a", "an", "one", "several", "many", "few", "a few", 
        "a couple of", "at least one", "a number of", "a lot of", 
        "a majority of", "a minority of", "certain", "most", "almost all", 
        "more than half", "half of", "less than half", "two-thirds", 
        "three-quarters", "dozens of", "hundreds of", "thousands of", 
        "millions of", "billions of", "about", "approximately", "various", 
        "plenty of", "somebody", "someone", "something", "somewhere", 
        "multiple", "varied", "part of", "enough", "much", 
        "a portion of", "particular", "certain individuals", "occasional"
    ]

    # Process each book
    for sentence_data in tqdm(data, desc="Processing books"):
            # Get the sentence and subject from the sentence data
        sentence = sentence_data.get('sentence', "")
        subjects = sentence_data.get('subjects', [])
        if subjects:
                # Assume the first subject is the main one (you can adjust if needed)
            subject_text = subjects[0].get('subject', "")
            if subject_text in excluded_pronouns:
                sentence_data['quantified_classification'] = "Neither"
            else:    
                # Compile the regular expressions using the current subject
                universal_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, universal_quantifiers)) + r')\b\s+\b' + re.escape(subject_text) + r'\b')
                existential_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, existential_quantifiers)) + r')\b\s+\b' + re.escape(subject_text) + r'\b')

                    # Categorize the sentence based on the subject
                classification = categorize_sentence_by_subject(sentence, subject_text, universal_pattern, existential_pattern)
                    # Add the classification to the sentence data
                sentence_data['quantified_classification'] = classification
        else:
                # If no subject is present, classify as "Neither"
            sentence_data['quantified_classification'] = "Neither"
    
    # Save the updated data to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

output_file = 'SPEECH_QUANTIFIERS.json'
process_json_file(generics_data, output_file)


Processing books: 100%|██████████| 360341/360341 [00:11<00:00, 30029.07it/s]
